In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [2]:
data = pd.read_csv('data/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [3]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [5]:
def weighted_random_recommendation(items_probabilities, n=5):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, probabilities. Сумма probabilities по всем товарам = 1
    """
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    items = np.array(items_probabilities.iloc[:, 0])
    probabilities = np.array(items_probabilities.iloc[:, 1])
    recs = np.random.choice(items, size=n, replace=False, p=probabilities)
    
    return recs.tolist()

In [6]:
items_probabilities = data.groupby('item_id')['sales_value'].sum().reset_index()
items_probabilities.head()

,item_id,sales_value
0,25671,20.94
1,26081,0.99
2,26093,1.59
3,26190,1.54
4,26355,1.98


In [7]:
items_probabilities['sales_value_log'] = np.log(items_probabilities['sales_value'])

C:\Users\infantium\anaconda3\lib\site-packages\pandas\core\series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [8]:
items_probabilities.loc[items_probabilities['sales_value_log'] == float('-inf'), 'sales_value_log'] = 0

In [9]:
items_probabilities.loc[items_probabilities['sales_value_log'] < 0, 'sales_value_log'] = np.abs(items_probabilities['sales_value_log'])

In [10]:
items_probabilities['probabilities'] = items_probabilities['sales_value_log'] / items_probabilities['sales_value_log'].sum()

In [11]:
items_probabilities['probabilities'].sum()

1.0

In [12]:
items_probabilities = items_probabilities.drop(columns=['sales_value', 'sales_value_log'])
items_probabilities.head()

,item_id,probabilities
0,25671,1.293695e-05
1,26081,4.274660e-08
2,26093,1.972377e-06
3,26190,1.836479e-06
4,26355,2.905382e-06


In [13]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items_probabilities, n=5))
result.head(2)

Wall time: 3.57 s


,user_id,actual,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[1086038, 961983, 5979546, 896513, 122952]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[3742933, 16057430, 8181246, 80773, 15629980]"


### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [14]:
result = pd.read_csv('predictions/predictions_basic.csv')
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,[ 821867 834484 856942 865456 889248 ...,"[8157117, 870291, 1846901, 891748, 1038560]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,[ 835476 851057 872021 878302 879948 ...,"[1000965, 10121560, 9878622, 13512901, 13039531]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [15]:
for i in range(result.shape[0]):
    result.loc[result['actual'] == result.loc[i]['actual'], 'actual'] = result.loc[i]['actual'].replace('\n',',').replace('[  ','[').replace('   ',', ').replace('  ',', ')

In [16]:
result.head(2)

,user_id,actual,random_recommendation,popular_recommendation,itemitem,cosine,tfidf,own_purchases
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[8157117, 870291, 1846901, 891748, 1038560]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1127831, 995242, 1098066]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 981760, 1127831, 1098066, 961554]","[1082185, 1029743, 995785, 1004906, 1081177]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[1000965, 10121560, 9878622, 13512901, 13039531]","[6534178, 6533889, 1029743, 6534166, 1082185]","[1082185, 981760, 1098066, 995242, 826249]","[1082185, 1098066, 981760, 826249, 883404]","[1082185, 981760, 1098066, 826249, 883404]","[1082185, 1098066, 6534178, 1127831, 1068719]"


In [17]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [18]:
columns = result.columns.tolist()[2:]
for col in columns:
    lists = []
    for i in range(result.shape[0]):
        pratk = precision_at_k(result.loc[i][f'{col}'].replace('[','').replace(']','').split(', '), result.loc[i]['actual'].replace('[','').replace(']','').split(', '), k=5)
        lists.append(pratk)
    res = np.array(lists).mean()
    print(f'{col}: {res}')

random_recommendation: 0.0002938295788442704
popular_recommendation: 0.12242899118511263
itemitem: 0.12438785504407443
cosine: 0.12820763956904993
tfidf: 0.1327130264446621
own_purchases: 0.15940254652301666


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ- {2000, 5000, 10000} товаров. Постройте график зависимости precision@5 от топ-*
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$. Постройте график зависимости precision@5 от $K$ 

In [19]:
# your_code